# Analítica Avanzada de Datos.
---

## Máquinas de Soporte Vectorial

Las máquinas de vectores soporte (SVM) son una clase particularmente potente y flexible de algoritmos supervisados tanto para la clasificación como para la regresión. En este notebook, exploraremos la intuición que hay detrás de las SVM y su uso en problemas de clasificación

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from scipy import stats

En lugar de modelizar cada clase, nos limitaremos a encontrar una línea o curva (en dos dimensiones) o un múltiple (en múltiples dimensiones) que separe las clases entre sí.
Como ejemplo de esto, consideremos el caso simple de una tarea de clasificación en la que las dos clases de puntos están bien separadas (véase la figura siguiente):

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=50, centers=2,
                  random_state=0, cluster_std=0.60)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');

Un clasificador discriminativo lineal intentaría trazar una línea recta que separara los dos conjuntos de datos, y crear así un modelo para la clasificación. Para datos bidimensionales como los que se muestran aquí, ésta es una tarea que podríamos hacer a mano. Pero tenemos un problema: hay más de una línea divisoria posible que puede discriminar perfectamente entre las dos clases

Podemos dibujar algunas de ellas de la siguiente manera:

In [ ]:
xfit = np.linspace(-1, 3.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plt.plot([0.6], [2.1], 'x', color='red', markeredgewidth=2, markersize=10)

for m, b in [(1, 0.65), (0.5, 1.6), (-0.2, 2.9)]:
    plt.plot(xfit, m * xfit + b, '-k')

plt.xlim(-1, 3.5);

Se trata de tres separadores muy diferentes que, sin embargo, discriminan perfectamente entre estas muestras. Dependiendo del que elijamos, a un nuevo punto de datos (por ejemplo, el marcado con una "X" en este gráfico) se le asignará una etiqueta diferente.

### Máquinas de vectores soporte: Maximización del margen

Las máquinas de soporte vectorial ofrecen una forma de mejorar esta situación. La intuición es la siguiente: en lugar de trazar simplemente una línea de anchura cero entre las clases, podemos dibujar alrededor de cada línea un margen de cierta anchura, hasta el punto más cercano. He aquí un ejemplo:

In [ ]:
xfit = np.linspace(-1, 3.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')

for m, b, d in [(1, 0.65, 0.33), (0.5, 1.6, 0.55), (-0.2, 2.9, 0.2)]:
    yfit = m * xfit + b
    plt.plot(xfit, yfit, '-k')
    plt.fill_between(xfit, yfit - d, yfit + d, edgecolor='none',
                     color='lightgray', alpha=0.5)

plt.xlim(-1, 3.5);

La línea que maximiza este margen es la que elegiremos como modelo óptimo.

### Ajuste de una máquina de soporte vectorial

Veamos el resultado de un ajuste real a estos datos: utilizaremos el clasificador de vectores de soporte (SVC) de Scikit-Learn para entrenar un modelo SVM en estos datos. Por el momento, vamos a utilizar un kernel lineal y establecer el parámetro C a un número muy grande:

In [ ]:
from sklearn.svm import SVC # "Support vector classifier"
model = SVC(kernel='linear', C=1E10)
model.fit(X, y)

Para visualizar mejor lo que está sucediendo aquí, vamos a crear una función de conveniencia rápida que trazará los límites de decisión SVM para nosotros:

In [ ]:
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, edgecolors='black',
                   facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(model);

Se trata de la línea divisoria que maximiza el margen entre los dos conjuntos de puntos. Observa que algunos de los puntos de entrenamiento tocan el margen: están rodeados por un círculo. Estos puntos son los elementos fundamentales de este ajuste; se conocen como **vectores de soporte** y dan nombre al algoritmo. En *Scikit-Learn*, las identidades de estos puntos se almacenan en el atributo *support_vectors_* del clasificador:

In [ ]:
model.support_vectors_

Una de las claves del éxito de este clasificador es que, para el ajuste, sólo importan las posiciones de los vectores de soporte; cualquier punto alejado del margen que esté en el lado correcto no modifica el ajuste. Técnicamente, esto se debe a que estos puntos no contribuyen a la función de pérdida utilizada para ajustar el modelo, por lo que su posición y número no importan mientras no crucen el margen.

Podemos ver esto, por ejemplo, si trazamos el modelo aprendido a partir de los primeros 60 puntos y los primeros 120 puntos de este conjunto de datos:

In [ ]:
def plot_svm(N=10, ax=None):
    X, y = make_blobs(n_samples=200, centers=2,
                      random_state=0, cluster_std=0.60)
    X = X[:N]
    y = y[:N]
    model = SVC(kernel='linear', C=1E10)
    model.fit(X, y)
    
    ax = ax or plt.gca()
    ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
    ax.set_xlim(-1, 4)
    ax.set_ylim(-1, 6)
    plot_svc_decision_function(model, ax)

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)
for axi, N in zip(ax, [60, 120]):
    plot_svm(N, axi)
    axi.set_title('N = {0}'.format(N))

En el lado izquierdo, vemos el modelo y los vectores de apoyo para 60 puntos de entrenamiento. En el lado derecho, hemos duplicado el número de puntos de entrenamiento, pero el modelo no ha cambiado: los tres vectores de soporte del lado izquierdo son los mismos que los del lado derecho. **Esta insensibilidad al comportamiento exacto de los puntos distantes es uno de los puntos fuertes del modelo SVM.**

### Más allá de los límites lineales: SVM de núcleo

SVM puede llegar a ser muy potente cuando se combina con los kernels. 
Para motivar la necesidad de núcleos, veamos algunos datos que no son linealmente separables:

In [ ]:
from sklearn.datasets import make_circles
X, y = make_circles(100, factor=.1, noise=.1)

clf = SVC(kernel='linear').fit(X, y)

plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(clf, plot_support=False);

Está claro que ninguna discriminación lineal podrá separar estos datos. Pero podemos pensar en cómo podríamos proyectar los datos en una dimensión superior de forma que un separador lineal fuera suficiente. Por ejemplo, una proyección sencilla que podríamos utilizar sería calcular una función de base radial (RBF) centrada en el grupo central:

In [ ]:
r = np.exp(-(X ** 2).sum(1))

Podemos visualizar esta dimensión adicional de los datos mediante un gráfico tridimensional:

In [ ]:
from mpl_toolkits import mplot3d

ax = plt.subplot(projection='3d')
ax.scatter3D(X[:, 0], X[:, 1], r, c=y, s=50, cmap='autumn')
ax.view_init(elev=20, azim=30)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('r');

Podemos ver que con esta dimensión adicional, los datos se vuelven trivialmente separables linealmente, dibujando un plano de separación en, digamos, r=0,7

En este caso, tuvimos que elegir y ajustar cuidadosamente nuestra proyección: si no hubiéramos centrado nuestra función de base radial en el lugar correcto, no habríamos obtenido unos resultados tan limpios y linealmente separables. En general, la necesidad de hacer una elección de este tipo es un problema: nos gustaría encontrar de algún modo automáticamente las mejores funciones de base para utilizar.

Una estrategia para ello es *calcular una función base centrada en cada punto del conjunto de datos y dejar que el algoritmo SVM examine los resultados*. Este tipo de transformación de la función base se conoce como **transformación kernel**, ya que se basa en una relación de similitud (o kernel) entre cada par de puntos.

Un problema potencial de esta estrategia -proyectar puntos en dimensiones- es que puede resultar muy intensiva desde el punto de vista computacional a medida que sea mayor. Sin embargo, gracias a un pequeño procedimiento conocido como el **truco del kernel**, se puede realizar un ajuste de los datos transformados por el kernel de forma implícita, es decir, sin tener que construir la representación dimensional completa del kernel. -de la proyección del kernel. Este truco de kernel está integrado en la SVM, y es una de las razones por las que el método es tan potente.

En Scikit-Learn, podemos aplicar SVM kernelizado simplemente cambiando nuestro kernel lineal a un kernel RBF, utilizando el hiperparámetro del modelo de kernel:

In [ ]:
clf = SVC(kernel='rbf', C=1E6)
clf.fit(X, y)

Utilicemos nuestra función previamente definida para visualizar el ajuste e identificar los vectores de soporte:

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(clf)
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
            s=300, lw=1, facecolors='none');

Utilizando esta máquina de soporte vectorial kernelizada, aprendemos un límite de decisión no lineal adecuado. Esta estrategia de transformación de kernel se utiliza a menudo en el aprendizaje automático para convertir métodos lineales rápidos en métodos no lineales rápidos, especialmente para modelos en los que se puede utilizar el truco del kernel.

### Ajuste de la SVM: Suavizar los márgenes

Nuestra discusión hasta ahora se ha centrado en conjuntos de datos muy limpios, en los que existe un límite de decisión perfecto. Pero, ¿qué ocurre si sus datos se mezclan? Por ejemplo, puede tener datos como éstos:

In [ ]:
X, y = make_blobs(n_samples=100, centers=2,
                  random_state=0, cluster_std=1.2)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');

Para manejar este caso, la implementación de SVM tiene un factor que "suaviza" el margen: es decir, permite que algunos de los puntos se cuelen en el margen si eso permite un mejor ajuste. La dureza del margen está controlada por un parámetro de ajuste, a menudo conocido como **C**. Para un **C** muy grande, el margen es duro, y los puntos no pueden estar en él. Con un **C** más pequeño, el margen es más suave y puede crecer hasta abarcar algunos puntos.

El gráfico de la figura siguiente muestra cómo el cambio de **C** afecta al ajuste final a través de la suavización del margen:

In [ ]:
X, y = make_blobs(n_samples=100, centers=2,
                  random_state=0, cluster_std=0.8)

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for axi, C in zip(ax, [10.0, 0.1]):
    model = SVC(kernel='linear', C=C).fit(X, y)
    axi.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
    plot_svc_decision_function(model, axi)
    axi.scatter(model.support_vectors_[:, 0],
                model.support_vectors_[:, 1],
                s=300, lw=1, facecolors='none');
    axi.set_title('C = {0:.1f}'.format(C), size=14)

El valor óptimo de C dependerá de su conjunto de datos, y deberá ajustar este parámetro utilizando la validación cruzada o un procedimiento similar

### Ejemplo: Reconocimiento facial

Como ejemplo de máquinas de soporte vectorial, veamos el problema del reconocimiento facial. Utilizaremos el conjunto de datos *Labeled Faces in the Wild*, que consta de varios miles de fotos recopiladas de diversos personajes públicos. Scikit-Learn incorpora un recuperador para el conjunto de datos:

In [ ]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)
print(faces.target_names)
print(faces.images.shape)

Vamos a trazar algunas de estas caras para ver con qué estamos trabajando:

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(8, 6))
for i, axi in enumerate(ax.flat):
    axi.imshow(faces.images[i], cmap='bone')
    axi.set(xticks=[], yticks=[],
            xlabel=faces.target_names[faces.target[i]])

Cada imagen contiene 62 × 47, es decir, unos 3.000 píxeles. Podríamos proceder simplemente utilizando el valor de cada píxel como característica, pero a menudo es más eficaz utilizar algún tipo de preprocesador para extraer características más significativas; aquí utilizaremos el análisis de componentes principales para extraer 150 componentes fundamentales para alimentar nuestro clasificador de máquina de vectores de soporte. Podemos hacerlo de la forma más sencilla empaquetando el preprocesador y el clasificador en un único canal:

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

pca = PCA(n_components=150, whiten=True,
          svd_solver='randomized', random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)

Para probar los resultados de nuestro clasificador, dividiremos los datos en un conjunto de entrenamiento y un conjunto de prueba:

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(faces.data, faces.target,
                                                random_state=42)

Por último, podemos utilizar la validación cruzada de búsqueda en cuadrícula para explorar combinaciones de parámetros. Aquí ajustaremos **C** (que controla la dureza del margen) y gamma (que controla el tamaño del núcleo de la función de base radial), y determinaremos el mejor modelo:

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid)

%time grid.fit(Xtrain, ytrain)
print(grid.best_params_)

Los valores óptimos caen hacia el centro de nuestra cuadrícula; si cayeran en los bordes, querríamos ampliar la cuadrícula para asegurarnos de que hemos encontrado el verdadero óptimo.

Ahora, con este modelo de validación cruzada, podemos predecir las etiquetas de los datos de prueba, que el modelo aún no ha visto:

In [ ]:
model = grid.best_estimator_
yfit = model.predict(Xtest)

Veamos algunas de las imágenes de prueba junto con sus valores previstos:

In [ ]:
fig, ax = plt.subplots(4, 6)
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(62, 47), cmap='bone')
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(faces.target_names[yfit[i]].split()[-1],
                   color='black' if yfit[i] == ytest[i] else 'red')
fig.suptitle('Predicted Names; Incorrect Labels in Red', size=14);

De esta pequeña muestra, nuestro estimador óptimo sólo etiquetó mal una cara (la cara de Bush en la fila inferior fue etiquetada erróneamente como Blair). Podemos hacernos una mejor idea del rendimiento de nuestro estimador utilizando el informe de clasificación, que enumera las estadísticas de recuperación etiqueta por etiqueta:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest, yfit,
                            target_names=faces.target_names))

También podríamos mostrar la matriz de confusión entre estas clases:

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
mat = confusion_matrix(ytest, yfit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d',
            cbar=False, cmap='Blues',
            xticklabels=faces.target_names,
            yticklabels=faces.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label');

Esto nos ayuda a hacernos una idea de qué etiquetas son susceptibles de ser confundidas por el estimador.

Para una tarea de reconocimiento facial en el mundo real, en la que las fotos no vienen previamente recortadas en bonitas cuadrículas, la única diferencia en el esquema de clasificación facial es la selección de características: sería necesario utilizar un algoritmo más sofisticado para encontrar las caras y extraer características que sean independientes de la pixelación. Para este tipo de aplicación, una buena opción es utilizar OpenCV, que, entre otras cosas, incluye implementaciones preentrenadas de las herramientas de extracción de características más avanzadas para imágenes en general y rostros en particular.

Mayor información: https://scikit-learn.org/stable/modules/svm.html